In [2]:
import pandas as pd
import numpy as np

#Load Dataset

In [3]:
balanced_p1 = pd.read_csv('/content/balanced_p1_reddit.csv')
random = pd.read_csv('/content/binary_random_reddit.csv')

#LSTM

In [4]:
balanced_p1.head()

,Unnamed: 0,T1,T2,class
0,0,I am not for or against any of the ideologies....,Child poverty causes a number of problems for ...,1.0
1,1,The Green and Libertarian parties have complet...,I am German and my opinion is based on the fla...,1.0
2,2,"As a Libertarian, I'm a firm believer that eve...","Today is Australia's National Holiday, celebra...",1.0
3,3,I am a young person (&lt;30 years) who has bee...,"Abortion, gun safety laws, school shootings, r...",1.0
4,4,"Let me start by saying, I was once just like y...","Now, he's a great guy, but politically... let'...",1.0


In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(balanced_p1['T1']+ balanced_p1['T2'], balanced_p1['class'], test_size=0.2, random_state=42)


In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, Embedding, LSTM, Dense, Dropout, Conv1D, MaxPooling1D, GlobalMaxPooling1D

In [11]:
# tokenization and padding
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)
max_length = max([len(seq) for seq in X_train_seq])
max_length = max([len(seq) for seq in X_test_seq])
X_train_padded = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=max_length, padding='post')

3530


In [17]:

# LSTM model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=32, input_length=max_length))
model.add(Conv1D(64, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(Conv1D(32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(32, return_sequences = True)))
model.add(Dropout(0.2))
model.add(MaxPooling1D(pool_size=5))
model.add(Dense(32, activation='tanh'))
model.add(GlobalMaxPooling1D())
model.add(Dense(2, activation='softmax'))  # 2 classes: generated by same method or not
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# train the model
model.fit(X_train_padded, y_train, epochs=15, batch_size=32)


Epoch 1/15
48/48 [==============================] - 20s 267ms/step - loss: 0.6903 - accuracy: 0.5553
Epoch 2/15
48/48 [==============================] - 13s 272ms/step - loss: 0.6166 - accuracy: 0.6678
Epoch 3/15
48/48 [==============================] - 12s 244ms/step - loss: 0.4947 - accuracy: 0.7592
Epoch 4/15
48/48 [==============================] - 11s 224ms/step - loss: 0.4033 - accuracy: 0.8066
Epoch 5/15
48/48 [==============================] - 11s 237ms/step - loss: 0.3899 - accuracy: 0.8309
Epoch 6/15
48/48 [==============================] - 11s 223ms/step - loss: 0.3113 - accuracy: 0.8632
Epoch 7/15
48/48 [==============================] - 11s 224ms/step - loss: 0.2438 - accuracy: 0.9086
Epoch 8/15
48/48 [==============================] - 10s 211ms/step - loss: 0.2394 - accuracy: 0.9138
Epoch 9/15
48/48 [==============================] - 10s 212ms/step - loss: 0.2155 - accuracy: 0.9211
Epoch 10/15
48/48 [==============================] - 9s 194ms/step - loss: 0.1520 - accurac

In [18]:
# tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_test)
loss, acc = model.evaluate(X_test_padded, y_test)

12/12 [==============================] - 2s 24ms/step - loss: 0.4732 - accuracy: 0.8158


In [19]:
X_test_padded.shape

(380, 3530)

In [20]:
X_train_padded.shape

(1520, 3530)

In [21]:
preds = model.predict(X_test_padded)
preds[1]


12/12 [==============================] - 2s 34ms/step


array([0.10229696, 0.8977031 ], dtype=float32)

In [22]:
preds_new = []
for i in preds:
  if(i[0]>i[1]):
    preds_new.append(0.0)
  else:
    preds_new.append(1.0)

In [23]:
preds_new[1]

1.0

In [24]:
y_test_array = np.array(y_test)
y_test_array[1]

1.0

In [26]:
from sklearn.metrics import f1_score,precision_score, recall_score, accuracy_score
f1 = f1_score(y_test, preds_new)
print("Balanced Scores")
print("F1 score = ",f1)
precision = precision_score(y_test, preds_new)
recall = recall_score(y_test, preds_new)
print("Precision = ", precision)
print("Recall = ", recall)
print("Accuracy = ", accuracy_score(y_test, preds_new))

Balanced Scores
F1 score =  0.7941176470588235
Precision =  0.8709677419354839
Recall =  0.7297297297297297
Accuracy =  0.8157894736842105
